# Fooling a Machine Learning Classifier using Adversarial Images 

###### Goals from this project:

* Practical experience with Tensorflow for implementing and trianing deep neural networks 
* Learn how to setup a ML classifier. Train model to predict MNIST handwritten digits with a 99.2% accuracy. 
* Learn how to create Adversary images 


In [3]:
# Start TensorFlow Interactive Session - builds a computation graph that runs with the graph 
import tensorflow as tf
sess = tf.InteractiveSession() 

In [4]:
#Load MNIST data stored in NumPy Arrays 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
# First start with softmax regression model with a single linear layer 

# Build computational graph by creating nodes for input images (x placeholder = 2d tensor of floats) and target output classes(y_ placeholder = 2d tensor for image and one-hot 10D vector)  
x = tf.placeholder(tf.float32, shape=[None, 784]) # 784 = number of pixels in a 28 x 28 MNIST image 
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [6]:
# Define model parameters (variables that are modified by the computation graph) which are weights W and biases b 

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [7]:
# Initalize the variables W and b so they can be used within a session

sess.run(tf.global_variables_initializer())

In [8]:
# Linear regression model
y = tf.matmul(x,W) + b

In [9]:
# Loss function : Cross entropy 
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [10]:
# Train the model

# steepest gradient descent algorithm with a 0.5 step size : computes gradients, parameter update steps, and applies update steps to parameters
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [11]:
# gradient descent updates parameters by repeatedly calling "train_step" 

# feed_dict replaces placeholders (x, y_) with training examples 
for _ in range(1000):
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [12]:
# Evaluate softmax regression model

# Find max entry along every row using argmax where y =  predicted label, y_ = true label
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [13]:
# Correct_prediction gives us list of booleans but we want a float 
# Cast to floats
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
# Accuracy of test data 
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9218


In [15]:
# Building a multilayer convolutional network 

# Create helper functions for weights and biases 
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [16]:
# Convolutions of stride 1 and no padding
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# Max pooling over 2x2 blocks
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [17]:
# First convolutional layer - computes 32 features(output) from the input for each 5x5 patch 

W_conv1 = weight_variable([5, 5, 1, 32])

# Bias vector with a component for each output channel
b_conv1 = bias_variable([32])

In [18]:
# Reshape x to a 4d tensor 
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [19]:
# Convolve x_image with weight tensor, add bias, then apply the ReLU function and max pool 
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

# reduce image size to 14x14
h_pool1 = max_pool_2x2(h_conv1) 

In [20]:
# Second Convolutional Layer - 64 features with each 5x5 patch
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [21]:
# Densely Connected Layer

# Image size is now 7x7, we want to add a fully-connected layer with 1024 neurons to process the entire image 
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# Reshape tensor from pooling layer into batch of vecctors, multiply by weight matrix, add bias and apply ReLU 
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [22]:
# Apply dropout before the readout layer to reduce overfitting 
# Dropout is on during training and off during testing 
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [23]:
# Readout layer 

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
# Train and Evaluate the model 

# We have ADAM optimizer instead of gradient descent optimizer here, and dropout rate is controlled by keep_prob
# During training, logging is added to every 100th iteration

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# tf.Session is used rather than tf.InteractiveSession to sepearte model specification and model fitting 
# "with" block in python: tf.session is destroyed once block is exited
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1
step 100, training accuracy 0.9
step 200, training accuracy 0.82
step 300, training accuracy 0.94
step 400, training accuracy 1


###### References 

[MNIST CNN classifier] (https://www.tensorflow.org/get_started/mnist/pros#deep-mnist-for-experts)

